# COPT and LOLP Calculator

## Initiate

### import and settting printer

In [ ]:
import itertools

import numpy as np
import pandas as pd

In [ ]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

### inputs

In [ ]:
# COPT input data path
CASE_NAME = 'COPT Case.csv'

In [ ]:
# demands peak load used for LOLP and EENS
DEMANDS = [590, 470, 585, 650, 660]

In [ ]:
# value of lost load
VOLL = 15_000_000

### pre-processing

In [ ]:
# extract csv
df = pd.read_csv(f'../data/{CASE_NAME}')

capacities = df['Capacity (MW)'].tolist()
outage_rates = df['FOR (Outage)'].tolist()
status = df['Status'].tolist()

pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,50,0.100,1
2,50,0.100,1
3,50,0.100,1
4,30,0.100,1
5,10,0.070,1
6,20,0.070,1
7,10,0.100,1
8,5,0.100,1
9,10,0.070,1
10,10,0.070,1


In [ ]:
# filter only available generator
generator_list = [[cap, 1-out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]

# make tables for each generator
# TODO: Tables already support derating, input data should also support it from excel
tables = [np.array([generator, [0, 1 - generator[1]]]) for generator in generator_list]

## COPT table

In [ ]:
table = tables[0].copy()  # copy to avoid modifying data
for table_ in tables[1:]:
    # faster than flatten + transpose
    table = np.hstack(((table[:, 0, None] + table_[:, 0]).reshape(-1,1),
                       (table[:, 1, None] * table_[:, 1]).reshape(-1,1)))

    # sort table
    table = table[(-table[:, 0]).argsort(),:]

    # combine duplicate
    table = np.array([[k,sum([x[1] for x in list(g)])] 
                    for k,g in itertools.groupby(table, lambda x:x[0])])
    
    # TODO: Implement resample, useful for big COPT table, triggered only if max table length achieve

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

df = pd.DataFrame(data=table,
                  columns=columns_name,
                  index=pd.RangeIndex(1, len(table) + 1, 1, name='No'),
                  )
df.to_csv(f'../results/COPT_{CASE_NAME}')
df.head(10)

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
No,,,,
1,875.000,0.024,1.000,0.024
2,870.000,0.020,0.976,0.045
3,865.000,0.042,0.955,0.086
4,860.000,0.030,0.914,0.117
5,855.000,0.051,0.883,0.168
6,850.000,0.036,0.832,0.204
7,845.000,0.050,0.796,0.254
8,840.000,0.034,0.746,0.287
9,835.000,0.040,0.713,0.328


## LOLP

In [ ]:
df = pd.read_csv(f'../results/COPT_{CASE_NAME}', index_col=0)
table = df.values

In [ ]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [ ]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in DEMANDS]
lolp

[0.0001933444358765,
 1.4799527806098814e-07,
 0.0001449366283941,
 0.0032895996560149,
 0.0050267079116176]

In [ ]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.1216


## EENS

In [ ]:
df = pd.read_csv(f'../results/COPT_{CASE_NAME}', index_col=0)
table = df.values

In [ ]:
def get_eens(capacity, individual_probability, demand):
    """
    format:
        capacity
        individual_probability
    """
    return sum(individual_probability
               * (capacity < demand)
               * (demand - capacity))

In [ ]:
eens = [get_eens(table[:,0], table[:,1], demand) for demand in DEMANDS]
eens

[0.0040499933019519825,
 2.4368428881072812e-06,
 0.0030832711225689917,
 0.07893220768003756,
 0.12389429805751778]

In [ ]:
print(f'Total EENS: {sum(eens):.4f}')

Total EENS: 0.2100


## Load Shedding Cost

In [ ]:
print(f'{VOLL * sum(eens):,.2f}')

3,149,433.11
